In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from helpers import *

In [ ]:
df = HSIDataSetDataFrame(pd.read_csv(DATA_DIR / "index.csv"))

In [ ]:
df.index

In [ ]:
img_num = 13
plt.imshow(df.images[img_num].hsi_calibrated[:, :, 110])

In [ ]:
from skimage import filters, morphology, measure, color

for i in range(49, 50):
    img_num = i
    threshold = 0.08
    min_area = 300
    image = df.images[img_num].hsi_calibrated[:, :, 110]
    gray_image = df.images[img_num].hsi_calibrated[:, :, 110]

    binary_mask = image[:580] > threshold
    dialated = morphology.dilation(binary_mask, morphology.disk(3))
    eroded = morphology.erosion(dialated, morphology.disk(3))

    # Perform morphological closing to join nearby regions
    closed = morphology.closing(
        eroded, morphology.disk(3)
    )  # Adjust disk size as needed

    # Remove small objects (regions with area < 100 pixels)
    filtered = morphology.remove_small_objects(closed, min_size=min_area)

    # use regioprops to get the area of each region
    regions = measure.regionprops(measure.label(filtered))
    areas = [region.area for region in regions]
    aspect_ratios = [
        region.major_axis_length / region.minor_axis_length for region in regions
    ]

    # Label connected components
    labeled_regions = measure.label(filtered)

    mask = np.zeros_like(labeled_regions, dtype=bool)
    # Loop through each region and create a mask for regions with aspect ratio <= 5
    for region in regions:
        aspect_ratio = region.major_axis_length / region.minor_axis_length
        if aspect_ratio > 5:  # Remove regions with aspect ratio > 5
            labeled_regions[labeled_regions == region.label] = 0

    # Assign random colours to each region
    colored_regions = color.label2rgb(labeled_regions, kind="overlay")

    # Display results
    # fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    # ax[0].imshow(image, cmap="gray")
    # ax[0].set_title(f"Fixed Threshold ({threshold})")
    # ax[0].axis("off")

    # ax[1].imshow(filtered, cmap="gray")
    # ax[1].set_title("After Removing Small Regions (<100 px)")
    # ax[1].axis("off")

    # ax[2].imshow(colored_regions)
    # ax[2].set_title("Segmented Regions with Unique Colours")
    # ax[2].axis("off")
    # np.unique(labeled_regions).shape[0]
    if np.unique(labeled_regions).shape[0] != 49:
        print(i, np.unique(labeled_regions).shape[0])
    # plt.show()

In [ ]:
def segment_image(image_num, threshold=0.08, min_area=300, plot=False):
    image = df.images[img_num].hsi_calibrated[:, :, 110]
    gray_image = df.images[img_num].hsi_calibrated[:, :, 110]

    binary_mask = image[:580] > threshold
    dialated = morphology.dilation(binary_mask, morphology.disk(3))
    eroded = morphology.erosion(dialated, morphology.disk(3))

    # Perform morphological closing to join nearby regions
    closed = morphology.closing(
        eroded, morphology.disk(3)
    )  # Adjust disk size as needed

    # Remove small objects (regions with area < 100 pixels)
    filtered = morphology.remove_small_objects(closed, min_size=min_area)

    # use regioprops to get the area of each region
    regions = measure.regionprops(measure.label(filtered))
    areas = [region.area for region in regions]
    aspect_ratios = [
        region.major_axis_length / region.minor_axis_length for region in regions
    ]

    # Label connected components
    labeled_regions = measure.label(filtered)

    mask = np.zeros_like(labeled_regions, dtype=bool)
    # Loop through each region and create a mask for regions with aspect ratio <= 5
    for region in regions:
        aspect_ratio = region.major_axis_length / region.minor_axis_length
        if aspect_ratio > 5:  # Remove regions with aspect ratio > 5
            labeled_regions[labeled_regions == region.label] = 0

    # Assign random colours to each region
    colored_regions = color.label2rgb(labeled_regions, bg_label=0, kind="overlay")

    # Display results
    if plot:
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].imshow(image, cmap="gray")
        ax[0].set_title(f"Fixed Threshold ({threshold})")
        ax[0].axis("off")

        ax[1].imshow(filtered, cmap="gray")
        ax[1].set_title("After Removing Small Regions (<100 px)")
        ax[1].axis("off")

        ax[2].imshow(colored_regions)

        for region in regions:
            ax[2].text(
                region.centroid[1], region.centroid[0], region.label, color="red"
            )

        ax[2].set_title("Segmented Regions with Unique Colours")
        ax[2].axis("off")
        plt.show()
    if np.unique(labeled_regions).shape[0] != 49:
        print(i, np.unique(labeled_regions).shape[0])

    return regions, labeled_regions


regions, labeled_regions = segment_image(i, threshold=0.1, plot=True)

In [ ]:
plt.imshow(labeled_regions == 4)


In [ ]:
from scipy.ndimage import center_of_mass

# Get unique region labels (excluding background 0)
region_labels = np.unique(labeled_regions)
region_labels = region_labels[region_labels > 0]  # Exclude background (0)

# Compute centroids of each region
centroids = np.array(
    center_of_mass(labeled_regions, labels=labeled_regions, index=region_labels)
)

# Sort by Y first (top to bottom), then by X (left to right)
sorted_indices = np.lexsort((centroids[:, 1], centroids[:, 0]))  # Sort by (Y, X)
new_labels = {old: new + 1 for new, old in enumerate(region_labels[sorted_indices])}

# Create new labeled image
renumbered_regions = np.copy(labeled_regions)
for old_label, new_label in new_labels.items():
    renumbered_regions[labeled_regions == old_label] = new_label

# renumbered_regions = renumbered_regions[renumbered_regions > 0]  # Remove background (0)

In [ ]:
np.unique(renumbered_regions)

In [ ]:
import numpy as np
from scipy.ndimage import center_of_mass


def sort_regions_grid(labeled_regions, y_threshold=5):
    region_labels = np.unique(labeled_regions)
    region_labels = region_labels[region_labels > 0]  # Exclude background (0)

    # Compute centroids
    centroids = np.array(
        center_of_mass(labeled_regions, labels=labeled_regions, index=region_labels)
    )

    # Sort centroids by Y-coordinates
    sorted_y_indices = np.argsort(centroids[:, 0])
    centroids = centroids[sorted_y_indices]
    region_labels = region_labels[sorted_y_indices]

    # Cluster regions into rows based on Y proximity
    rows = []
    current_row = [region_labels[0]]

    for i in range(1, len(region_labels)):
        if centroids[i, 0] - centroids[i - 1, 0] > y_threshold:
            rows.append(current_row)
            current_row = []
        current_row.append(region_labels[i])

    rows.append(current_row)  # Append last row

    # Sort each row by X-coordinates
    sorted_labels = []
    for row in rows:
        row_centroids = centroids[np.isin(region_labels, row)]
        sorted_x_indices = np.argsort(row_centroids[:, 1])
        sorted_labels.extend(np.array(row)[sorted_x_indices])

    # Renumber labels
    new_labels = {old: new + 1 for new, old in enumerate(sorted_labels)}
    renumbered_regions = np.copy(labeled_regions)
    for old_label, new_label in new_labels.items():
        renumbered_regions[labeled_regions == old_label] = new_label

    return renumbered_regions


# Usage
renumbered_regions = sort_regions_grid(labeled_regions, y_threshold=25)

In [ ]:
plt.imshow(renumbered_regions == 0, cmap="turbo")
for i in range(1, 49):
    region = renumbered_regions == i
    # compute the center of mass
    center = np.array(center_of_mass(region))
    # print(f"Region {i}: {center}")
    # plt.imshow(renumbered_regions == i)
    # plot centre
    # plt.scatter(center[1], center[0], color="red", s=100, marker="x")
    plt.text(
        center[1],
        center[0],
        str(i),
        color="white",
        fontsize=14,
        fontdict={"weight": "bold"},
        ha="center",
        va="center",
    )
plt.show()

In [ ]:
for i in range(10, 11):
    segment_image(i, threshold=0.1, plot=True)

In [ ]:
regions = segment_image(i, threshold=0.1, plot=True)

In [ ]:
def extract_masks(labeled_regions):
    region_labels = np.unique(labeled_regions)
    region_labels = region_labels[region_labels > 0]  # Exclude background (0)

    masks = {
        label: (labeled_regions == label).astype(np.uint8) for label in region_labels
    }
    return masks  # Dictionary {label: mask}


masks = extract_masks(renumbered_regions)

In [ ]:
masks[1]

In [ ]:
# plt.imshow(df.images[img_num].hsi_calibrated[masks[1]])a


In [ ]:
masks[1].shape
# plt.imshow(masks[1], cmap="gray")

In [ ]:
df.images[img_num].hsi_calibrated.shape

In [ ]:
img = df.images[img_num].hsi_calibrated[:580]

In [ ]:
img.shape

In [ ]:
masks[1]

In [ ]:
res = img * masks[1][:, :, np.newaxis]
plt.imshow(res[:, :, 10], cmap="gray")
res.shape

In [ ]:
plt.imshow(df.images[img_num].hsi_calibrated[:, :, 100])